# Large Format plots

This example makes plots like the ones seen in this image: 

![Data analysis with pen and paper tools in Plaça del Sol - Barcelona](https://live.staticflickr.com/4490/24368448418_d602723a10_h.jpg)

## Load data

In [ ]:
import scdata as sc
from scdata._config import config

config.log_level='DEBUG'
config.framework='jupyterlab'

In [ ]:
test = sc.Test(name='EXAMPLE')
await test.load()

## Make plots

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["NOISE_A"],
                  "subplot": 1}
        }

test.ts_uplot(traces=traces)

In [ ]:
# DIR for export
from os.path import exists
from os import mkdir

savedir = '~/Desktop/pdf-scdata'
if not exists(savedir): mkdir(savedir)

### Metrics total

In [ ]:
imgs = 1
pages = []

metric = "NOISE_A"
ylabel_i = "Noise (dBA)"
yrange_i = [20, 60]
paginate_every = 5
# min_date = '2021-06-12'
# max_date = '2021-06-14'
sandbox = False
title = "VenicAir"
paper = "A2"
margin = 15/25.4 # in inches

# Get paper width and size for resolution and proportions 
if int(paper[1])%2 == 0:
    height = 1/pow(2.0,(int(paper[1])/2))*1000/pow(2,0.25)
    width = 1/pow(2.0,(int(paper[1])/2))*1000*pow(2,0.25)
else:
    height = 1.0/pow(2.0,((int(paper[1])+1.0)/2.0))*1000.0*pow(2.0, 0.25)
    width = 1.0/pow(2,((int(paper[1])-1.0)/2))*1000/pow(2.0, 0.25)
print(f'Paper size (w, h): {width, height} mm')
print(f'Paper size (w, h): {width/25.4, height/25.4} in')

height_plot = height/25.4-margin
width_plot = width/25.4

print(f'Plot size (w, h): {width_plot, height_plot} in')

vec_filename = f'{paper}_export_{metric}_*.svg'
pdf_filename = f'{paper}_output_{metric}.pdf'
n_pages = len(test.devices)//paginate_every + 1

def setBold(txt): return r"$\bf{" + str(txt) + "}$"
def setItalic(txt): return r"$\it{" + str(txt) + "}$"

for page in range(0, n_pages):
    traces = {}
    yrange = {}
    ylabel = {}
    
    for device in test.devices:
        if device.id in pages: continue
        pages.append(device.id)      
        print (device.id)
        print (metric)
        traces [str(imgs)] = {"devices": device.id,
                              "channel": metric,
                              "subplot": imgs
                          }

        yrange [imgs] = yrange_i
        if device.source.type == 'csv': continue
        if 'json' in vars(device.handler):
            owner = device.handler.json.owner.username
        else:
            owner = ''
        ylabel [imgs] = setBold(ylabel_i) + '\n\n' + str(device.id) + '\n' + setItalic(f"user: {owner}")

        imgs += 1
        if imgs == paginate_every + 1: 
            imgs = 1
            break

    options = {
                "show": True,
                "frequency": '5Min',
                "clean_na": False,
                # "max_date": max_date,
                # "min_date": min_date
                }

    formatting = {"xlabel": "Date",
                  "ylabel": ylabel,
                  "yrange": yrange,
                  # "xrange": {1: [min_date, max_date]},
                  "hspace": 0.25,                  
                  "title": None,
                  "sharex":True,
                  "grid": True,
                  "legend": False,
                  "height": height_plot,
                  "width": width_plot,
                  'fontsize': 12,
                  "suptitle_factor": 0.92,
                  "suptitle_x": 0.125,
                  "title": setBold(title) + '\n' + min_date + ' to ' + max_date,
                  "title_fontsize": 20,
                  "fontname": 'Liberation Sans',
                  "title_loc": "left",
                  "style": "seaborn-whitegrid",
                  "palette": "gray"
                  }

    f = test.ts_plot(traces = traces, options = options, formatting = formatting);
    if f is not None:
        f.savefig(f"{savedir}/{vec_filename.replace('*', str(page))}", format = 'svg')
        print (f'Page {page} saved')
    else:
        print (f'Could not save page {page}')
    
    if sandbox == True: break

### User total


Make sure that the settings (y-ranges, dates, are applicable)

In [ ]:
# User total
imgs = 1
pages = []

metrics = ["PM_1", "NOISE_A", "TEMP", "HUM"]
ylabels_i = ["PM1 (ug/m3)", "NOISE (dBA)", "Temperature (degC)", "Humidity (\%rh)"]
yranges_i = [[0, 90], [20, 90], [15, 45], [0, 100]]
paginate_every = 5
min_date = '2021-09-28 10:00:00'
max_date = '2021-09-29 18:00:00'
sandbox = False
title = "VenicAir"
paper = "A1"
margin = 15/25.4 # in inches

# Get paper width and size for resolution and proportions 
if int(paper[1])%2 == 0:
    height = 1/pow(2.0,(int(paper[1])/2))*1000/pow(2,0.25)
    width = 1/pow(2.0,(int(paper[1])/2))*1000*pow(2,0.25)
else:
    height = 1.0/pow(2.0,((int(paper[1])+1.0)/2.0))*1000.0*pow(2.0, 0.25)
    width = 1.0/pow(2,((int(paper[1])-1.0)/2))*1000/pow(2.0, 0.25)
print(f'Paper size (w, h): {width, height} mm')
print(f'Paper size (w, h): {width/25.4, height/25.4} in')

height_plot = height/25.4-margin
width_plot = width/25.4

print(f'Plot size (w, h): {width_plot, height_plot} in')

n_pages = len(test.devices)

def setBold(txt): return r"$\bf{" + str(txt) + "}$"
def setItalic(txt): return r"$\it{" + str(txt) + "}$"

for device in test.devices:
    imgs = 1
    traces = {}
    yrange = {}
    ylabel = {}
    
    vec_filename = f'{paper}_export_{device}.svg'
    pdf_filename = f'{paper}_output_{device}.pdf'
    
    pages.append(device)
    d = test.devices[device].api_device.devicejson
    
    for metric in metrics:
        
        traces [str(imgs)] = {"devices": device,
                              "channel": metric,
                              "subplot": imgs
                          }

        yrange [imgs] = yranges_i[metrics.index(metric)]
        ylabel [imgs] = setBold(ylabels_i[metrics.index(metric)])

        imgs += 1

    options = {
                "show": True,
                "frequency": '5Min',
                "clean_na": None,
                "max_date": max_date,
                "min_date": min_date
                }

    formatting = {"xlabel": "Date",
                  "ylabel": ylabel,
                  "yrange": yrange,
                  "xrange": {1: [min_date, max_date]},
                  "hspace": 0.25,                  
                  "title": None,
                  "sharex":True,
                  "grid": True,
                  "legend": False,
                  "height": height_plot,
                  "width": width_plot,
                  'fontsize': 12,
                  "suptitle_factor": 0.92,
                  "suptitle_x": 0.125,
                  "title":  setBold('Sensor: ') + d['name'] + '\n' + setItalic('by ') + setItalic({d['owner']['username'].replace('_', '')}),
                  "title_fontsize": 20,
                  "fontname": 'Liberation Sans',
                  "title_loc": "left",
                  "style": "seaborn-whitegrid",
                  "palette": "gray"
                  }

    f = test.ts_plot(traces = traces, options = options, formatting = formatting);
    if f is not None:
        f.savefig(f"{savedir}/{vec_filename}", format = 'svg')
        print (f'Page {page} saved')
    else:
        print (f'Could not save page {page}')

## Make PDFs

From now on is all svg. Needs inkscape installed to do this automatically:

In [ ]:
#!cd $savedir && inkscape * --export-area-drawing --batch-process --export-type=pdf --export-type=$pdf_filename
!cd $savedir && inkscape * --export-area-page --batch-process --export-type=pdf

In [ ]:
!cd $savedir && open *.pdf